# COGS 189 Final Project 

**Team Name**  
Argus

**Team Members**  
Xiaolong Huang	(xih002@ucsd.edu),  
Bohan Lei		(blei@ucsd.edu),  
Mingze Xu		(m6xu@ucsd.edu),  
Weijia Zeng		(wezeng@ucsd.edu)     

**Project Goal**
- To fully understand and implement the vanilla Common Spatial Pattern (CSP) algorithm for EEG pattern recognition.
- To experiment with different types of Common Spatial Pattern (CSP) and Linear Discriminant Analysis (LDA) algorithms in motor imagery signal pattern classification.
- To improve motor imagery signal pattern classification accuracy by different CSP and LDA hyperparameter configurations.

**Sources**
- Data source: https://github.com/bregydoc/bcidatasetIV2a
- Data explanation: https://www.bbci.de/competition/iv/desc_2a.pdf
- CSP paper: https://ieeexplore.ieee.org/document/4408441
- SACSP paper: https://cogsci.ucsd.edu/~desa/Winter_conference_on_bci_2022.pdf

## Section 1: Setup

In [1]:
import numpy as np
from scipy.linalg import eigh
from scipy.signal import butter, sosfiltfilt, sosfreqz
import matplotlib.pyplot as plt

## Section 2: Data Import

In [22]:
n_subject = 9

data_dir = './data/'
data_prefix = 'A'
data_suffix = '.npz'
data_type = {'train':'T', 'test':'E'}
data_path = data_dir + data_prefix + '{subject:02d}{type_:s}' + data_suffix

In [23]:
raw_train_subject = []
raw_test_subject = []

for subject_num in range(1, n_subject+1):
    train_path = data_path.format(subject=subject_num, type_=data_type['train'])
    test_path  = data_path.format(subject=subject_num, type_=data_type['test'])
    raw_train_subject.append(np.load(train_path))
    raw_test_subject.append(np.load(test_path))

In [24]:
class_code = {'left':769, 'right':770, 'foot':771, 'tongue':772}

In [25]:
# globally
fs = 250                                    # sampling frequency

# within in each epoch
cue_start = 2.                              # cue starts at
cue_end = 3.25                              # cue ends at
smr_start = 3.                              # smr starts after
smr_end = 6.                                # smr ends after
cue_start_offset = round(cue_start * fs)    # the first cue sample is at 
cue_end_offset   = round(cue_end * fs)      # the last cue sample is at
smr_start_offset = round(smr_start * fs)    # the first smr sample is after
smr_end_offset   = round(smr_end * fs)      # the last smr sample is after

In [41]:
train_subject = []
for raw_data in raw_train_subject: 
    signal = raw_data['s'].T
    cue_class = raw_data['etyp']
    cue_position = raw_data['epos']
    
    X_train = {}
    
    for name, code in class_code.items():
        epoch_start = cue_position[cue_class==code] - cue_start_offset
        
        epochs = []
        for i in range(len(epoch_start)):
            epochs.append(signal[:, epoch_start[i]+smr_start_offset:epoch_start[i]+smr_end_offset])
        epochs = np.stack(epochs)
        
        X_train[name] = epochs
    
    train_subject.append(X_train)

In [44]:
for item in raw_test_subject[0].files:
    print(item)
    print(raw_test_subject[0][item])

s
[[ 11.23046875 -27.24609375   6.10351562 ...   3.41796875  45.41015625
  -12.6953125 ]
 [  9.5703125  -22.55859375   7.91015625 ...   3.90625     45.41015625
  -11.71875   ]
 [ 10.69335938 -26.12304688   5.61523438 ...  10.7421875   52.24609375
   -2.44140625]
 ...
 [  2.68554688   3.02734375   3.41796875 ...   0.9765625   -5.37109375
   -8.7890625 ]
 [  6.49414062   5.859375     5.859375   ...   9.765625    -2.9296875
   -6.34765625]
 [ 14.20898438   9.86328125  12.6953125  ...  13.18359375   8.30078125
    5.37109375]]
etyp
[[32766]
 [  276]
 [32766]
 [  277]
 [32766]
 [ 1072]
 [32766]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [  783]
 [  768]
 [ 1023]
 [  783]
 [  768]
 [ 1023]
 [  783]
 [  768]
 [  783]
 [  768